In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType

import warnings
warnings.filterwarnings('ignore')

In [2]:
#.master("spark://10.84.128.47:7077") \

spark = SparkSession\
    .builder\
    .appName('BD_G10')\
    .config("hive.metastore.uris", "thrift://10.0.0.0:9083")\
    .enableHiveSupport()\
    .getOrCreate()

/usr/local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [4]:
spark.sql("CREATE TABLE IF NOT EXISTS g10.raw_data_textfile (instant float,id_activity int,heart_rate int,imu_hand_1 float,imu_hand_2 float,imu_hand_3 float,imu_hand_4 float,imu_hand_5 float,imu_hand_6 float,imu_hand_7 float,imu_hand_8 float,imu_hand_9 float,imu_hand_10 float,imu_hand_11 float,imu_hand_12 float,imu_hand_13 float,imu_hand_14 float,imu_hand_15 float,imu_hand_16 float,imu_hand_17 float,imu_chest_1 float,imu_chest_2 float,imu_chest_3 float,imu_chest_4 float,imu_chest_5 float,imu_chest_6 float,imu_chest_7 float,imu_chest_8 float,imu_chest_9 float,imu_chest_10 float,imu_chest_11 float,imu_chest_12 float,imu_chest_13 float,imu_chest_14 float,imu_chest_15 float,imu_chest_16 float,imu_chest_17 float,imu_ankle_1 float,imu_ankle_2 float,imu_ankle_3 float,imu_ankle_4 float,imu_ankle_5 float,imu_ankle_6 float,imu_ankle_7 float,imu_ankle_8 float,imu_ankle_9 float,imu_ankle_10 float,imu_ankle_11 float,imu_ankle_12 float,imu_ankle_13 float,imu_ankle_14 float,imu_ankle_15 float,imu_ankle_16 float,imu_ankle_17 float) ROW FORMAT DELIMITED FIELDS TERMINATED BY ' ' STORED AS TEXTFILE")

DataFrame[]

In [5]:
spark.sql("LOAD DATA INPATH 'hdfs://10.84.128.47:9000/aulas/BD_G10/subject101.dat' OVERWRITE INTO TABLE g10.raw_data_textfile")

DataFrame[]

In [6]:
spark.sql("select * from g10.raw_data_textfile limit 1").show()

+-------+-----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+
|instant|id_activity|heart_rate|imu_hand_1|imu_hand_2|imu_hand_3|imu_hand_4|imu_hand_5|imu_hand_6|imu_hand_7|imu_hand_8|imu_hand_9|imu_hand_10|imu_hand_11|imu_hand_12|imu_hand_13|imu_hand_14|imu_hand_15|imu_hand_16|imu_hand_17|imu_chest_1|imu_chest_2|imu_chest_3|imu_chest_4|imu_chest_5|imu_chest_6|imu_chest_7|imu_chest_8|imu_chest_9|imu_chest_10|

In [11]:
raw_data = spark.sql("SELECT instant, id_activity, heart_rate, (imu_hand_1+imu_hand_2+imu_hand_3+imu_hand_4+imu_hand_5+imu_hand_6+imu_hand_7+imu_hand_8+imu_hand_9+imu_hand_10+imu_hand_11+imu_hand_12+imu_hand_13+imu_hand_14+ imu_hand_15+ imu_hand_16+ imu_hand_17) as Sum_IMU_Hand, (imu_chest_1+imu_chest_2+imu_chest_3+imu_chest_4+imu_chest_5+imu_chest_6+imu_chest_7+imu_chest_8+imu_chest_9+imu_chest_10+imu_chest_11+imu_chest_12+imu_chest_13+imu_chest_14+ imu_chest_15+ imu_chest_16+ imu_chest_17) as Sum_IMU_Chest, (imu_ankle_1+imu_ankle_2+imu_ankle_3+imu_ankle_4+imu_ankle_5+imu_ankle_6+imu_ankle_7+imu_ankle_8+imu_ankle_9+imu_ankle_10+imu_ankle_11+imu_ankle_12+imu_ankle_13+imu_ankle_14+ imu_ankle_15+ imu_ankle_16+ imu_ankle_17) as Sum_IMU_Ankle FROM g10.raw_data_textfile")
raw_data.write.format("orc").saveAsTable("g10.raw_data_inicial")
spark.sql("select * from g10.raw_data_inicial limit 1").show()

+-------+-----------+----------+------------+-------------+-------------+
|instant|id_activity|heart_rate|Sum_IMU_Hand|Sum_IMU_Chest|Sum_IMU_Ankle|
+-------+-----------+----------+------------+-------------+-------------+
|1777.06|         13|       129|   87.451584|    40.021736|    27.250303|
+-------+-----------+----------+------------+-------------+-------------+



In [13]:
spark.sql("DROP TABLE if exists g10.raw_data_textfile")

DataFrame[]

In [14]:
dataset_final = spark.sql("SELECT id_activity, avg(heart_rate) over(order by instant rows between 4 preceding and current row) avg_values_heart_rate, min(heart_rate) over(order by instant rows between 4 preceding and current row) min_values_heart_rate, max(heart_rate) over(order by instant rows between 4 preceding and current row) max_values_heart_rate,std(heart_rate) over(order by instant rows between 4 preceding and current row) std_values_heart_rate,avg(Sum_IMU_Hand) over(order by instant rows between 4 preceding and current row) avg_values_imu_hand, min(Sum_IMU_Hand) over(order by instant rows between 4 preceding and current row) min_values_imu_hand, max(Sum_IMU_Hand) over(order by instant rows between 4 preceding and current row) max_values_imu_hand,std(Sum_IMU_Hand) over(order by instant rows between 4 preceding and current row) std_values_imu_hand,avg(Sum_IMU_Chest) over(order by instant rows between 4 preceding and current row) avg_values_imu_chest, min(Sum_IMU_Chest) over(order by instant rows between 4 preceding and current row) min_values_imu_chest, max(Sum_IMU_Chest) over(order by instant rows between 4 preceding and current row) max_values_imu_chest,std(Sum_IMU_Chest) over(order by instant rows between 4 preceding and current row) std_values_imu_chest,avg(Sum_IMU_Ankle) over(order by instant rows between 4 preceding and current row) avg_values_imu_ankle, min(Sum_IMU_Ankle) over(order by instant rows between 4 preceding and current row) min_values_imu_ankle, max(Sum_IMU_Ankle) over(order by instant rows between 4 preceding and current row) max_values_imu_ankle,std(Sum_IMU_Ankle) over(order by instant rows between 4 preceding and current row) std_values_imu_ankle FROM g10.raw_data_inicial GROUP BY instant, id_activity, heart_rate, Sum_IMU_Ankle,Sum_IMU_Chest, Sum_IMU_Hand")

In [15]:
label_activity = {1: 'lying', 2: 'sitting', 3: 'standing', 4: 'walking', 5: 'running', 6: 'cycling', 7: 'Nordic walking', 9: 'watching TV', 10: 'computer work', 11: 'car driving', 12: 'ascending stairs', 13: 'descending stairs', 16: 'vacuum cleaning', 17: 'ironing', 18: 'folding laundry', 19: 'house cleaning', 20: 'playing soccer', 24: 'rope jumping', 0: 'other'}

schema = StructType([
    StructField("activity", StringType(), True), StructField("avg_values_heart_rate", FloatType(), True),
    StructField("min_values_heart_rate", IntegerType(), True), StructField("max_values_heart_rate", IntegerType(), True), 
    StructField("std_values_heart_rate", FloatType(), True), StructField("avg_values_imu_hand", FloatType(), True),
    StructField("min_values_imu_hand", FloatType(), True), StructField("max_values_imu_hand", FloatType(), True),
    StructField("std_values_imu_hand", FloatType(), True), StructField("avg_values_imu_chest", FloatType(), True),
    StructField("min_values_imu_chest", FloatType(), True), StructField("max_values_imu_chest", FloatType(), True),
    StructField("std_values_imu_chest", FloatType(), True), StructField("avg_values_imu_ankle", FloatType(), True),
    StructField("min_values_imu_ankle", FloatType(), True), StructField("max_values_imu_ankle", FloatType(), True),
    StructField("std_values_imu_ankle", FloatType(), True)
])

In [16]:
df2 = dataset_final.rdd.map(lambda row: (label_activity[row.id_activity], row.avg_values_heart_rate, row.min_values_heart_rate, row.max_values_heart_rate, row.std_values_heart_rate, row.avg_values_imu_hand, row.min_values_imu_hand, row.max_values_imu_hand, row.std_values_imu_hand, row.avg_values_imu_chest, row.min_values_imu_chest, row.max_values_imu_chest, row.std_values_imu_chest, row.avg_values_imu_ankle, row.min_values_imu_ankle, row.max_values_imu_ankle, row.std_values_imu_ankle)).toDF(schema=schema)
df2.limit(1).show()
df2.createOrReplaceTempView("dataset")

+--------+---------------------+---------------------+---------------------+---------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|activity|avg_values_heart_rate|min_values_heart_rate|max_values_heart_rate|std_values_heart_rate|avg_values_imu_hand|min_values_imu_hand|max_values_imu_hand|std_values_imu_hand|avg_values_imu_chest|min_values_imu_chest|max_values_imu_chest|std_values_imu_chest|avg_values_imu_ankle|min_values_imu_ankle|max_values_imu_ankle|std_values_imu_ankle|
+--------+---------------------+---------------------+---------------------+---------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--

In [17]:
spark.sql("CREATE TABLE IF NOT EXISTS g10.dataset_final STORED AS ORC as SELECT * FROM dataset")

DataFrame[]

In [18]:
spark.stop()